### VDJ recombination statistics

Assesses correlations between VDJ recombination and cluster assignment.

In [1]:
import os
import copy
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pandas as pd
from Bio import SeqIO
from antpack import SingleChainAnnotator, SequenceScoringTool

aligner = SingleChainAnnotator(chains=["H", "K", "L"], scheme = "imgt")

if "notebooks" in os.getcwd():
    current_dir = os.path.join(os.getcwd(), "..", "..")

score_tool = SequenceScoringTool()

os.chdir(current_dir)
tt_dir = [f for f in os.listdir() if f.startswith("train_test_data_immunogenicity")]
if len(tt_dir) != 1:
    raise ValueError("Data not downloaded yet, or multiple versions of data downloaded.")

tt_dir = tt_dir[0]

os.chdir(os.path.join(current_dir, tt_dir, "vdj_statistics_eval"))

In [2]:
def get_contingency_table(v1, v2):
    """Generates a 2d contingency table for two lists of nominal variables."""
    v1_dict, v2_dict = {}, {}
    v1_counter, v2_counter = 0, 0
    
    for v1_item, v2_item in zip(v1, v2):
        if v1_item not in v1_dict:
            v1_dict[v1_item] = v1_counter
            v1_counter += 1
        if v2_item not in v2_dict:
            v2_dict[v2_item] = v2_counter
            v2_counter += 1

    contingency_table = np.zeros((len(v1_dict), len(v2_dict)), dtype=np.int32)
    for v1_item, v2_item in zip(v1, v2):
        contingency_table[v1_dict[v1_item], v2_dict[v2_item]] += 1

    return contingency_table

In [3]:
heavy = pd.read_csv("heavy_cab_vdj_assigned.csv.gz")
light = pd.read_csv("light_cab_vdj_assigned.csv.gz")
#Sequences that contain nonstandard AAs (e.g. *) cannot be scored by SAM and hence have nan scores.
#There are only a few dozen of these.
heavy = heavy.loc[~heavy.SAM_cluster_weighted.isna(),:]
light = light.loc[~light.SAM_cluster_weighted.isna(),:]

/tmp/ipykernel_195402/1648468156.py:1: DtypeWarning: Columns (29,36,42) have mixed types. Specify dtype option on import or set low_memory=False.
  heavy = pd.read_csv("heavy_cab_vdj_assigned.csv.gz")


In [4]:
heavy["v_family"] = [v.split("-")[0] for v in heavy.v_gene]
light["v_family"] = [v.split("-")[0] for v in light.v_gene]

In [5]:
hvfamily = get_contingency_table(heavy.v_family.tolist(), heavy.SAM_cluster_weighted.astype(np.int32).tolist())
hvgene = get_contingency_table(heavy.v_gene.tolist(), heavy.SAM_cluster_weighted.astype(np.int32).tolist())

In [6]:
from scipy.stats.contingency import association

print(association(hvfamily))
print(association(hvgene))

0.9998671417124277
0.7799582860214327


In [7]:
lvfamily = get_contingency_table(light.v_family.tolist(), light.SAM_cluster_weighted.astype(np.int32).tolist())
lvgene = get_contingency_table(light.v_gene.tolist(), light.SAM_cluster_weighted.astype(np.int32).tolist())

In [8]:
print(association(lvfamily))
print(association(lvgene))

0.9976540045213792
0.9571492039456065


In [9]:
hjgene = get_contingency_table(heavy.j_gene.tolist(), heavy.SAM_cluster_weighted.astype(np.int32).tolist())
ljgene = get_contingency_table(light.j_gene.tolist(), light.SAM_cluster_weighted.astype(np.int32).tolist())

In [10]:
print(association(hjgene))
print(association(ljgene))

0.418588663165082
0.5320739391830961
